<a href="https://colab.research.google.com/github/zonnesmit/zonnesmit.github.io/blob/main/retrieval-models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval Models and Evaluation



In this notebook, you will evaluate results ranking on a test collection. First, you'll compute the mean average precision of a baseline BM25 model. Then you'll implement a query-likelihood retrieval model based on the same inverted index.

This notebook uses the [Pyserini](http://pyserini.io/) library, a Python interface to [Anserini](http://anserini.io) and thus to [Lucene](https://lucene.apache.org/), a widely-used open-source search engine. This library is written and maintained by Jimmy Lin and his colleagues at the University of Waterloo.


We start by installing the python interface. Since it calls the underlying Lucene search engine, we make sure we point to an appropriate Java installation. If you don't have Java 21, this would need to be changed.

In [1]:
## Uncomment the following code to install Java 21 on Colab
!apt-get install openjdk-21-jre-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-21-openjdk-amd64/bin/java
!java -version

openjdk version "21.0.4" 2024-07-16
OpenJDK Runtime Environment (build 21.0.4+7-Ubuntu-1ubuntu222.04)
OpenJDK 64-Bit Server VM (build 21.0.4+7-Ubuntu-1ubuntu222.04, mixed mode, sharing)


In [2]:
!pip install pyserini
# You can change this to gpu if you have one.
# It's a pyserini dependency, but we won't need it until the next assignment.
!pip install faiss-cpu

You can use the `LuceneSearcher` to search over an index. We can initialize the searcher with a pre-built index, which Pyserini will automatically download if it hasn't already:

In [3]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher.from_prebuilt_index('robust04')

lucene-inverted.disk45.20240803.36f7e3.tar.gz: 1.66GB [00:18, 93.9MB/s]                           


Now we can search for a query and inspect the results:

In [4]:
hits = searcher.search('black bear attacks', 1000)

# Prints the first 10 hits
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')

 1 LA092790-0015   7.06680
 2 LA081689-0039   6.89020
 3 FBIS4-16530     6.61630
 4 LA102589-0076   6.46450
 5 FT932-15491     6.25090
 6 FBIS3-12276     6.24630
 7 LA091090-0085   6.17030
 8 FT922-13519     6.04270
 9 LA052790-0205   5.94060
10 LA103089-0041   5.90650


The `IndexReaderUtils` class provides various methods to read the index directly. For example, we can fetch a raw document from the index given its `docid`:

In [5]:
from pyserini.index import IndexReader
from IPython.core.display import display, HTML

reader = IndexReader.from_prebuilt_index('robust04')

doc = reader.doc('LA092790-0015').raw()
display(HTML('<div style="font-family: Times New Roman; padding-bottom:10px">' + doc + '</div>'))

Note that the result is exactly the same as displaying the hit contents above. Given the raw text, we can obtain its analyzed form (i.e., tokenized, stemmed, stopwords removed, etc.). Here we show the first ten tokens:

In [6]:
analyzed = reader.analyze(doc)
analyzed[0:10]

['date',
 'p',
 'septemb',
 '27',
 '1990',
 'thursdai',
 'ventura',
 'counti',
 'edit',
 'p']

The index also stores the raw document vector, which we can obtain as a Python dictionary of analyzed terms to counts (i.e., term frequency).
For brevity, we only look at terms that appear more than once:

In [7]:
doc_vector = reader.get_document_vector('LA092790-0015')
{ k: v for (k, v) in doc_vector.items() if v >1 }

{'been': 11,
 'habitat': 2,
 'year': 3,
 'jenk': 4,
 'mountain': 3,
 'rees': 3,
 'would': 2,
 'debusscher': 3,
 'near': 3,
 'hungri': 3,
 'past': 2,
 'especi': 2,
 'wildlif': 2,
 'lion': 3,
 'ventura': 4,
 'anim': 9,
 'forest': 2,
 'two': 3,
 'seen': 3,
 'attack': 3,
 'black': 2,
 'i': 2,
 'food': 3,
 'counti': 4,
 'rural': 3,
 "they'r": 5,
 'sever': 3,
 'california': 2,
 'advis': 2,
 'parch': 2,
 'area': 4,
 'she': 3,
 'month': 4,
 'deer': 3,
 'famili': 2,
 'we': 2,
 'peopl': 2,
 'just': 3,
 'live': 2,
 'drought': 4,
 'ha': 3,
 'he': 2,
 'hi': 2,
 'septemb': 2,
 'yard': 2,
 'elsewher': 2,
 'natur': 2,
 'out': 4,
 'lot': 2,
 'have': 16,
 'leav': 2,
 'more': 3,
 'off': 2,
 'ojai': 3,
 'report': 4,
 'bobcat': 4,
 'coyot': 10,
 'few': 2,
 'bear': 4,
 'depart': 2,
 'author': 3,
 'dry': 2,
 'vallei': 4,
 'who': 2,
 'game': 2,
 'resid': 2,
 'hous': 3,
 'cat': 3,
 'sai': 2,
 'said': 19,
 'park': 2,
 'on': 2,
 'offici': 5,
 'wild': 3}

## Evaluating Ranked Results

We can load some standard evaluation sets such as Robust04, which contains 250 queries, or "topics" as the TREC conferences call them.

In [8]:
from pyserini.search import get_topics
topics = get_topics('robust04')
print(f'{len(topics)} queries total')

250 queries total


The topics are in a dictionary, whose keys are integers uniquely identifying each query. Each topic contains the following fields:

* `title`: TREC's term for the brief query a user might actually type;
* `description`: a longer form of the query in the form of a complete sentence; and
* `narrative`: a description of what the user is looking for and what kinds of results would be relevant or non-relevant.

In [9]:
topics[301]

{'narrative': 'A relevant document must as a minimum identify the organization and the type of illegal activity (e.g., Columbian cartel exporting cocaine). Vague references to international drug trade without identification of the organization(s) involved would not be relevant.',
 'description': 'Identify organizations that participate in international criminal activity, the activity, and, if possible, collaborating organizations and the countries involved.',
 'title': 'International Organized Crime'}

For the purpose of your experiments, we'll divide them into a development and test set.

In [10]:
dev_topics = {k:topics[k] for k in list(topics.keys())[:125]}
test_topics = {k:topics[k] for k in list(topics.keys())[125:]}

Now, we'll fetch the relevance judgments for the Robust04 queries, which TREC calls "qrels".



In [11]:
from urllib.request import urlopen

qfile = 'https://github.com/castorini/anserini-tools/blob/63ceeab1dd94c1221f29b931d868e8fab67cc25c/topics-and-qrels/qrels.robust04.txt?raw=true'
qrels = []
for line in urlopen(qfile):
  qid, round, docid, score = line.strip().split()
  qrels.append([int(qid), 0, docid.decode('UTF-8'), int(score)])
#qrels = [line.strip().split() for line in urlopen(qfile)]

Each record in the qrel contains four fields:

1. the numeric identifier of the query;
2. the round of relevance feedback, which is here always 0;
3. the identifier of a documennt that has been judged; and
4. the relevance score of that document.

In Robust04, all relevance judgments are binary, i.e., 1 or 0. Note that not all non-relevant documents are recorded. The qrel file only contains those documents the annotators actually looked at; the vast majority of documents in the collection have not been judged. In IR evaluation, we assume that unannotated documents are non-relevant.

In [12]:
qrels[0:10]

[[301, 0, 'FBIS3-10082', 1],
 [301, 0, 'FBIS3-10169', 0],
 [301, 0, 'FBIS3-10243', 1],
 [301, 0, 'FBIS3-10319', 0],
 [301, 0, 'FBIS3-10397', 1],
 [301, 0, 'FBIS3-10491', 1],
 [301, 0, 'FBIS3-10555', 0],
 [301, 0, 'FBIS3-10622', 1],
 [301, 0, 'FBIS3-10634', 0],
 [301, 0, 'FBIS3-10635', 0]]

## Computing Mean Average Precision

The Robust04 collection uses binary relevance judgments and usually has multiple relevant results for each query. It is thus common to use **mean average precision** (MAP) to evaluate retrieval performance on it. Remember from class that MAP adds the precision at the position of each _relevant_ document in a ranked list and then divides by the total number of relevant documents. So that we don't have to scan through the entire collection, we usually evaluate MAP at some maximum rank value, such as 100 or 1000. We simply stop scanning at that maximum rank.

As we saw above, you should pass a query string (the `title` of a topic) and the desired number of results to the `search` method of the `searcher` object.

In [13]:
hits = searcher.search(dev_topics[355]['title'], 1000)
[(hit.docid, hit.score) for hit in hits[0:10]]

[('FBIS4-20436', 11.349300384521484),
 ('FBIS3-23683', 9.126500129699707),
 ('FBIS3-21238', 8.309300422668457),
 ('FBIS4-44915', 7.935699939727783),
 ('FBIS4-20602', 7.6006999015808105),
 ('FBIS4-47382', 7.529600143432617),
 ('FT943-1589', 7.480100154876709),
 ('LA071789-0059', 7.451399803161621),
 ('FBIS4-22145', 7.215099811553955),
 ('FBIS4-44667', 7.106500148773193)]

For this assignment, evaluate MAP@1000 for the list of `test_topics` we created above. You should process the `qrels` data to find the relevant results for each query.

In [21]:
## TODO [35 points]: Compute MAP@1000 for test_topics

# Step 1: Organize the qrels data into a dictionary for fast lookup
qrel_dict = {}
for qid, _, docid, score in qrels:
    if qid not in qrel_dict:
        qrel_dict[qid] = {}
    qrel_dict[qid][docid] = score

def calculate_average_precision(retrieved_docs, relevant_docs):
    """Compute Average Precision for a single query."""
    if not relevant_docs:
        return 0.0

    num_relevant = 0
    precision_sum = 0.0

    for rank, (docid, score) in enumerate(retrieved_docs, start=1):
        if docid in relevant_docs and relevant_docs[docid] == 1:
            num_relevant += 1
            precision_sum += num_relevant / rank

    return precision_sum / len(relevant_docs) if relevant_docs else 0.0

# Step 2: Calculate MAP@1000
map_score = 0.0
num_queries = 0

for qid, topic in test_topics.items():
    relevant_docs = qrel_dict.get(qid, {})
    if not relevant_docs:
        continue  # skip if no relevance judgments for this query

    # Retrieve top 1000 documents
    retrieved_docs = [(hit.docid, hit.score) for hit in searcher.search(topic['title'], 1000)]
    avg_precision = calculate_average_precision(retrieved_docs, relevant_docs)

    map_score += avg_precision
    num_queries += 1

# Step 3: Average over all queries to get MAP@1000
map_at_1000 = map_score / num_queries if num_queries > 0 else 0.0
print(f'MAP@1000: {map_at_1000:.4f}')


MAP@1000: 0.0132


## Implementing a Query Likelihood Model

The default `LuceneSearcher` in pyserini uses a BM25 model. In this part of the assignment, you'll interact directly with the index to implement a query likelihood retrieval model.  For more details on pyserini's index API, [see the documentation](https://github.com/castorini/pyserini/blob/master/docs/usage-indexreader.md).

We've already created an `IndexReader` for the Robust04 collection and assigned it to `reader`. Remember that we can pass documents or queries through the index's predefined analysis pipeline, a series of operations like stemming, stopword removal, etc.

In [22]:
reader.analyze('Are there black bear attacks there?')

['black', 'bear', 'attack']

For most retrieval models, including BM25 and query likelihood, we'll want to know the overall statistics for the terms we retrieve from the index. By default, pyserini will do stemming and other "analysis" steps on the terms you give it. But since we're already working with stemmed queries, we'll tell it not to perform this analysis now.

In [23]:
# Just checking that our term is already in its index form.
print(reader.analyze('bear'))

df, cf = reader.get_term_counts('bear', analyzer=None)
print(f'The document frequency is {df} and the collection frequency is {cf}.')

['bear']
The document frequency is 16429 and the collection frequency is 23545.


You might also want to know summary statistics about the collection.

In [24]:
reader.stats()

{'total_terms': 174540872,
 'documents': 528030,
 'non_empty_documents': 528030,
 'unique_terms': 923436}

Now let's traverse the postings for a term.

In [25]:
posting_list = reader.get_postings_list('bear', analyzer=None)
print(f'There are {len(posting_list)} postings.')

posting = posting_list[100]
print(f'Each posting contains a document ID ({posting.docid}), term frequency ({posting.tf}), and position list ({posting.positions}).')

print(f'searcher has a method for turning internal integer IDs like {posting.docid} into external IDs like {searcher.doc(posting.docid).docid()}.')

There are 16429 postings.
Each posting contains a document ID (3405), term frequency (1), and position list ([461]).
searcher has a method for turning internal integer IDs like 3405 into external IDs like LA031889-0009.


If, instead of mapping terms to documents, you want to use a document name to get term information, see above for the `get_document_vector` method.

Now you should have all the information you need to implement a query likelihood retrieval model. Use Dirichlet smoothing with $\mu = 550$.

In [35]:
## TODO [50 points]: Implement a query likelihood model with Dirichlet smoothing.
## For a given query, index reader, and constant k, return the k documents with the highest query likelihoods.
## Think about what data s would let you keep the top k.

def query_likelihood(query, reader, k, mu=550):
    # Analyze the query to get the list of terms
    query_terms = reader.analyze(query)

    # Get collection statistics
    collection_stats = reader.stats()
    total_terms = collection_stats['total_terms']

    # Calculate collection frequency P(t|C) for each query term
    term_collection_probs = {}
    for term in query_terms:
        _, cf = reader.get_term_counts(term, analyzer=None)
        term_collection_probs[term] = cf / total_terms

    # Dictionary to store scores for each document
    doc_scores = {}

    # Go through each term's postings
    for term in query_terms:
        posting_list = reader.get_postings_list(term, analyzer=None)
        if not posting_list:
            continue

        for posting in posting_list:
            docid = posting.docid
            # Convert internal doc ID to external doc ID
            external_docid = reader.convert_internal_docid_to_external(docid)
            tf = posting.tf

            # Use external doc ID to get document vector
            doc_length = sum(reader.get_document_vector(external_docid).values())

            # Calculate P(t|d) for each term in the query using Dirichlet smoothing
            p_td = (tf + mu * term_collection_probs[term]) / (doc_length + mu)

            # Accumulate scores for each document by summing log(p_td) (using natural log approximation)
            if docid in doc_scores:
                doc_scores[docid] += -2.3025851 * (1 - p_td)  # approximation for log(p_td)
            else:
                doc_scores[docid] = -2.3025851 * (1 - p_td)

    # Sort documents by score in descending order and get the top k
    top_k_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)[:k]

    # Convert internal doc IDs to external IDs and return with scores
    return [(reader.convert_internal_docid_to_external(docid), score) for docid, score in top_k_docs]

Now you can reuse some of your evaluation code for mean average precision from above.

In [36]:
## TODO [15 points]: Compute the MAP@1000 for the test set you used above to evaluate BM25.

def mean_average_precision(test_topics, qrels_dict, reader, k=1000):
    total_map = 0
    num_queries = len(test_topics)

    for qid, topic in test_topics.items():
        relevant_docs = set(docid for docid, score in qrels_dict.get(qid, []) if score == 1)

        if not relevant_docs:
            continue

        ranked_docs = query_likelihood(topic['title'], reader, k)

        num_relevant = 0
        avg_precision = 0
        for rank, (docid, _) in enumerate(ranked_docs, start=1):
            docid = str(docid)  # Ensure docid is passed as a string
            if docid in relevant_docs:
                num_relevant += 1
                avg_precision += num_relevant / rank

        if num_relevant > 0:
            avg_precision /= len(relevant_docs)

        total_map += avg_precision

    return total_map / num_queries if num_queries > 0 else 0

test_map = mean_average_precision(test_topics, qrels_dict, reader, k=1000)
print(f'MAP@1000 for the test set: {test_map:.5f}')



AttributeError: 'IndexReader' object has no attribute 'convert_internal_docid_to_external'